In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
df_train=pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')

df_test=pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
print("Number os missing data trainset:", df_train.isna().any().sum())
print("Number os missing data testset:", df_test.isna().any().sum())

In [ ]:
df_train.describe()

In [ ]:
y_train=df_train['target']
X = df_train.drop(['id','target'], axis=1)

In [ ]:
encoder = OneHotEncoder(categories = 'auto')
y_enc = encoder.fit_transform(y_train.values.reshape(X.shape[0],1)).toarray()

In [ ]:
k=y_train.values
y=list(map(lambda x:x[-1],k))

In [ ]:
grid_param = {
    'objective': ['multiclass'],
    'class_weight':['balanced'],
    'learning_rate':[0.2,0.5],
    "num_leaves":[31,33],
    'metric': ['multi_logloss'],
    "n_estimators": [100],
    "reg_alpha":[0.1,0.3],
    "reg_lambda":[0.1,0.3],
    'verbosity':[ -1]
}
model = lgb.LGBMClassifier(silent=False)
bst_gs_cv = GridSearchCV(
            model,
            grid_param,
            cv = KFold(n_splits=5, shuffle=True),
            scoring = 'neg_mean_squared_log_error'
            )

In [ ]:
bst_gs_cv.fit(X,y,early_stopping_rounds=50,
             eval_set= [(X, y)])

In [ ]:
lr=bst_gs_cv.best_estimator_
lr.fit(X,y)

In [ ]:
df_test=pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
X_test = df_test.drop(['id'], axis=1)

In [ ]:
Y_t=lr.predict_proba(X_test)

In [ ]:
ans_df = pd.DataFrame(Y_t,columns = ['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])

In [ ]:
ans_df['id']=df_test['id'].values

In [ ]:
output = ans_df[['id','Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']]
output.to_csv('submission.csv', index=False)

In [ ]:
output